In [61]:
# Imports
import os, sys
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D


# to enable local imports
module_path = os.path.abspath('../../code')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
from machine_learning_load_data import loadOnlineEEGdata

D:\Masterthesis\thesis_eeg\code


In [2]:
tf.__version__

'2.1.0'

In [3]:
keras.__version__

'2.2.4-tf'

In [30]:
# Load some online EEG Data
eegData, freqData, entropyData = loadOnlineEEGdata(dirPath='../../../EEG_Data/eeg_data_online', splitData=True)

X_train_eeg, y_train_eeg, X_test_eeg, y_test_eeg = eegData
X_train_freq, y_train_freq, X_test_freq, y_test_freq = freqData
X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy = entropyData

# reshape
X_train_freq = X_train_freq.reshape(X_train_freq.shape[0], X_train_freq.shape[2])
X_test_freq = X_test_freq.reshape(X_test_freq.shape[0], X_test_freq.shape[2])

X_train_entropy = X_train_entropy.reshape(X_train_entropy.shape[0], X_train_entropy.shape[2])
X_test_entropy = X_test_entropy.reshape(X_test_entropy.shape[0], X_test_entropy.shape[2])


Loading Online EEG Data from ../../../EEG_Data/eeg_data_online ...
EEG Data Shape:
(5024, 512, 40) (5024,) (2154, 512, 40) (2154,)
Freq Data Shape:
(1008, 1, 1200) (1008,) (432, 1, 1200) (432,)
Entropy Data Shape:
(5024, 1, 200) (5024,) (2154, 1, 200) (2154,)


In [3]:
def TrainEvaluateNet(x_train, y_train, x_test, y_test):

    #Convert data set to numpy array of type float32
    #x_train = np.asarray(x_train, np.float32)
    #y_train = np.asarray(y_train, np.float32)
    #x_test = np.asarray(x_test, np.float32)
    #y_test = np.asarray(y_test, np.float32)

    #Number of neurons
    neurons_num = int((2/3) * x_train.shape[2])

    #Initialize neural net
    #output of layer = keras.layers.Dense(number of neurons, activation function, name)(input of layer)
    inputs = keras.Input(shape=(x_train.shape[1], x_train.shape[2]), name='input_points')
    x = keras.layers.Dense(neurons_num, activation='relu', name='L1')(inputs)
    x = keras.layers.Dense(neurons_num, activation='relu', name='L2')(x)
    outputs = keras.layers.Dense(2, activation='softmax', name='output_point')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='fatigue_classifier')

    #Set optimizer, loss function and metrics to track
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #binary_crossentropy

    #Summary of the neural net
    #model.summary()
    
    #Train neural net
    model.fit(x_train, y_train, epochs=3)

    #Evaluate neural net
    model.evaluate(x_test, y_test)

    #Predict best route
    return model

In [27]:
freq_conv = convertDataSet(X_train_freq, channels=30)
freq_conv.shape

(1008, 40, 30, 1)

In [31]:
def convertDataSet(dataset, channels):
    #converts the frequency dataset from (1008,1,1200) (samples, placeholder, channels + values) in (1008,40,30,1) (samples, channels, values, placeholder)
    newdataset = np.zeros((int(dataset.shape[0]), int((dataset.shape[2]/channels)), int(channels), 1))
    for sample in range(dataset.shape[0]):
        for channel in range(0, dataset.shape[2], channels):
            for values in range(channels):
                newdataset[sample, int(channel/30), values, 0] = dataset[sample, 0, channel + values]
    
    newdataset = np.asarray(newdataset, np.float32)
    return newdataset
    
def trainEvaluateNet(x_train, y_train, x_test, y_test):

    #Settings
    batch_size = 64
    num_classes = 2
    epochs = 20
    neurons_num = int((2/3) * x_train.shape[2]) #Number of neurons

    #converting training dataset
    #x_train = convertDataSet(x_train, 30)
    #x_test = convertDataSet(x_test, 30)

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    #Initialize neural net
    model = Sequential()
    model.add(Conv1D(32, kernel_size=10, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Conv1D(64, kernel_size=1, activation='relu'))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(neurons_num, activation='relu'))
    model.add(Dense(neurons_num, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax', name='last_dense')) 

    model.summary()
    
    #Set optimizer, loss function and metrics to track
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(0.01), metrics=['accuracy'])
    #model.compile(loss=keras.losses.squared_hinge, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

    #Summary of the neural net
    #model.summary()
    
    #Train neural net
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    #         validation_split=0.2)

    #Evaluate neural net
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    #Predict best route
    return model

In [62]:
def trainEasyNet(x_train, y_train, x_test, y_test, batch_size=64, num_classes=2, epochs=5):
    
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    neurons_num = int((2/3) * x_train.shape[1]) #Number of neurons
    
    # define the keras model
    model = Sequential()
    model.add(Dense(int(neurons_num*2), input_dim=x_train.shape[1], activation='relu'))
    model.add(Dense(neurons_num, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(int(neurons_num/2), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', name='last_dense'))
    
    #model.summary()
    
    # compile the keras model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(0.01), metrics=['accuracy'])
    # fit the keras model on the dataset
    
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
    
    #Evaluate neural net
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [63]:
# frequency data
model = trainEasyNet(X_train_freq, y_train_freq, X_test_freq, y_test_freq, epochs=10, batch_size=128)

ValueError: Input 0 is incompatible with layer global_average_pooling1d_1: expected ndim=3, found ndim=2

In [59]:
# entropy data
model = trainEasyNet(X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy, epochs=20, batch_size=80)

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 266)               53466     
_________________________________________________________________
dense_58 (Dense)             (None, 133)               35511     
_________________________________________________________________
dense_59 (Dense)             (None, 66)                8844      
_________________________________________________________________
dropout_19 (Dropout)         (None, 66)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 33)                2211      
_________________________________________________________________
last_dense (Dense)           (None, 2)                 68        
Total params: 100,100
Trainable params: 100,100
Non-trainable params: 0
_______________________________________________

In [42]:
def ApplyNet(input_data, model):
    prediction = model.predict([input_data]) 
    return prediction



#model = trainEvaluateNet(X_train_freq, y_train_freq, X_test_freq, y_test_freq)
#ApplyNet(model)


Epoch 1/10
1008/1008 [==============================] - 0s 471us/step - loss: 3.2995 - accuracy: 0.5258
Epoch 2/10
1008/1008 [==============================] - 0s 357us/step - loss: 0.6450 - accuracy: 0.6230
Epoch 3/10
1008/1008 [==============================] - 0s 330us/step - loss: 0.5557 - accuracy: 0.7222
Epoch 4/10
1008/1008 [==============================] - 0s 333us/step - loss: 0.4948 - accuracy: 0.7788
Epoch 5/10
1008/1008 [==============================] - 0s 337us/step - loss: 0.4586 - accuracy: 0.7956
Epoch 6/10
1008/1008 [==============================] - 0s 342us/step - loss: 0.3816 - accuracy: 0.8313
Epoch 7/10
1008/1008 [==============================] - 0s 347us/step - loss: 0.3965 - accuracy: 0.8264
Epoch 8/10
1008/1008 [==============================] - 0s 333us/step - loss: 0.2886 - accuracy: 0.8730
Epoch 9/10
1008/1008 [==============================] - 0s 337us/step - loss: 0.3082 - accuracy: 0.8532
Epoch 10/10
1008/1008 [==============================] - 0s 372u

In [ ]:
#model = TrainEvaluateNet(X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy)
#ApplyNet(model)
